# Лабораторная работа №2. Библиотеки NumPy и Pandas.

## NumPy

In [88]:
import numpy as np

### Задание 1

Реализуйте функции для перемножения матриц с использованием библиотеки NumPy и без неё (то есть на чистом Python). Сравните время работы функций при помощи команды `%%time`.

#### Vanilla python

In [89]:
def multiplication(m1, m2):
    # Функцию взял у себя же с 1-го курса
    # Проверка на совпадение кол-ва строк и столбцов для перемножения
    if len(m1[0]) != len(m2):
        raise AttributeError('Матрицы не возможно перемножить')

    zipped_m2 = zip(*m2)
    zipped_m2 = list(zipped_m2)
    # Зипом "склеиваем" элементы для итерации генератора
    return [[sum(ij*ji for ij, ji in zip(m1_r, m2_c)) for m2_c in zipped_m2] for m1_r in m1]

In [90]:
%%time
m1 = [[1,2,3],[1,2,3],[1,2,3]]
m2 = [[3,2,1], [3,2,1], [3,2,1]]

multiplication(m1,m2)

CPU times: user 120 µs, sys: 2 µs, total: 122 µs
Wall time: 131 µs


[[18, 12, 6], [18, 12, 6], [18, 12, 6]]

#### NumPy

In [91]:
%%time
m1 = np.array([[1,2,3], [1,2,3], [1,2,3]])
m2 = np.array([[3,2,1], [3,2,1], [3,2,1]])

m1.dot(m2)

CPU times: user 89 µs, sys: 2 µs, total: 91 µs
Wall time: 97.5 µs


array([[18, 12,  6],
       [18, 12,  6],
       [18, 12,  6]])

#### Вывод

По идее np должен быть быстрее, чем чистый python, но в моем случае он чаще уступал чистому python.

Скорее всего такой эффект из-за небольших матриц, представленных для перемножения.

### Задание 2

Напишите функцию, которая по данной последовательности $\{A_i\}_{i=1}^n$ строит последовательность $S_n$, где $S_k = \frac{A_1 + ... + A_k}{k}$.
Аналогично -- с помощью библиотеки NumPy и без нее. Сравните скорость, объясните результат.

#### Vanilla python

In [92]:
def avg_section(arr: list, start_k = 0):
    # arr - исходный массив
    # start_k - точка отсчёта
    # через enumerate() короче, чем через range(len())
    for k, v in enumerate(arr[start_k:]):
        yield sum(arr[:k])/(k+1)

In [93]:
test_arr = [i for i in range(1000)]
%time [i for i in avg_section(test_arr)]
# %time - ломает запись в переменную
[i for i in avg_section(test_arr)][-5:]

CPU times: user 5.48 ms, sys: 0 ns, total: 5.48 ms
Wall time: 5.37 ms


[496.50100401606426,
 497.0010030090271,
 497.501002004008,
 498.001001001001,
 498.501]

#### NumPy

In [94]:
test_arr = np.array([i for i in range(1000)])
k = 999

In [95]:
new_arr = np.empty(test_arr.size)
%time for i, el in enumerate(new_arr): new_arr[i] = test_arr[:i].sum()/(i+1)
new_arr[-5:]

CPU times: user 8.49 ms, sys: 0 ns, total: 8.49 ms
Wall time: 10.1 ms


array([496.50100402, 497.00100301, 497.501002  , 498.001001  ,
       498.501     ])

#### Вывод

Не знаю почему numpy в среднем медленнее, чем чистый python.
Может я как-то не правильно его использую.

Многократные прогоны на одних данных дают следующие результаты:

- чистый python: от 6 мс до 10 мс
- numpy: от 5 до 13 мс

В 8 из 10 попытках чистый python был быстрее, либо равен по времени выполнения numpy.

Использование генератора при тесте чистого python не должно дать преимущества, тк количество операций не сокращается.

### Задание 3

Пусть задан некоторый массив $X$. Надо построить новый массив, где все элементы с нечетными индексами требуется заменить на число $a$ (если оно не указано, то на 1). Все четные элементы исходного массива нужно возвести в куб и записать в обратном порядке относительно позиций этих элементов. Массив $X$ при этом должен остаться без изменений. В конце требуется слить массив $X$ с преобразованным $X$ и вывести в обратном порядке.

In [96]:
def morph_arr(arr: list, a=1):
    # Я не знаю, как такую дичь кратко написать с использованием numpy
    calc = [el ** 3 if i % 2 == 0 else a for i, el in enumerate(arr)]
    calc = [j for i in zip(calc[::2], calc[::-2]) for j in i]
    return list(arr + calc)[::-1]

In [97]:
# arr = [-10, -3, 0, 1, 4, 5, 6, 7, 8, 9]
arr = [-2, 13, 23, 2, 54, -1, -5, 100]
%time morph_arr(arr)

CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 15.3 µs


[1, -125, 1, 157464, 1, 12167, 1, -8, 100, -5, -1, 54, 2, 23, 13, -2]

### Задание 4

Реализуйте <a href="https://habr.com/ru/post/128454/" target="_blank">метод Монте-Карло</a> для вычисления числа $\pi$. Напишите две версии: с использованием NumPy и без. Сравните время работы. Вычислите значение числа $\pi$ с точностью до второго знака после запятой.

#### Vanilla python

In [98]:
from random import random
rng = 10000000
%time 4 * sum(1 for _ in range(rng) if random()**2 + random()**2 < 1) / rng

CPU times: user 3.13 s, sys: 3.72 ms, total: 3.14 s
Wall time: 3.14 s


3.1424472

#### NumPy

In [99]:
%%time
rndm = np.random.RandomState()
x = rndm.uniform(size=rng)
y = rndm.uniform(size=rng)
r = x*x + y*y
print(4.0 * np.count_nonzero(r < 1) / rng)

3.1416676
CPU times: user 198 ms, sys: 209 ms, total: 407 ms
Wall time: 406 ms


#### Вывод
У numpy значительный прирост производительности, при кратном увеличении количества элементов выборки